In [1]:

import webscraping
import pandas as pd
import fitz
import pandas as pd
from pathlib import Path
import re
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests

#Here is where we set the keyword list to be used across all sources
#This will be used as a list of search terms to look through in order to find sustainability-related headlines
keywords = [
    "sustainability",
    "sustainable",
    "greenwashing",
    "carbon-neutral",
    "renewable",
    "eco-friendly",
    "pollution",
    "emissions",
    "recycling",
    "green",
    "cruelty-free",
    "zero-waste",
    "carbon",
    "energy",
    "climate",
    "solar",
    "power",
    "water",
    "wind",
    "waste",
    "gas",
    "electricity",
    "fossil",
    "clean",
    "renewable energy",
    "climate change",
    "fossil fuels",
    "green energy",
    "solar farm",
    "greenhouse gas",
    "clean energy",
    "sustainable consumer",
    "sustainable developement",
    "sustainable consumption"
]

IndentationError: expected an indented block after function definition on line 107 (webscraping.py, line 108)

In [ ]:
#Journals

#get pdfs from folder on your computer, you will need to change this when running the code for yourself
location = '/Users/famema17/Downloads/ucla25-26/fall25/pic16B/project/data/'

#this code uses the funtion pdf_scraper from webscraping.py to search these journals for the keywords

pdf_path = location + 'anneJournal.pdf'
df = pdf_scraper(pdf_path, keywords)
df

pdf_path2 = location + 'andersonJournal.pdf'
df2 = pdf_scraper(pdf_path2, keywords)
df2

pdf_path3 = location + 'zhangJournal.pdf'
df3 = pdf_scraper(pdf_path3, keywords)
df3

pdf_path4 = location + 'carpenterJournal.pdf'
df4 = pdf_scraper(pdf_path4, keywords)
df4

pdf_path5 = location + 'horichJournal.pdf'
df5 = pdf_scraper(pdf_path5, keywords)
df5

pdf_path6 = location +'loresJournal.pdf'
df6 = pdf_scraper(pdf_path6, keywords)
df6

pdf_path7 = location + 'shettyJournal.pdf'
df7 = pdf_scraper(pdf_path7, keywords)
df7

pdf_path8 = location + 'veraJournal.pdf'
df8 = pdf_scraper(pdf_path8, keywords)
df8


#combine the individual data into one dataframe
df_journals = journalpdf_concat(df, df2, df3, df4, df5, df6, df7,df8)
df_journals.head()

In [ ]:
#News Articles

#ap news
dfs_with_all_keywords = pd.DataFrame()
for word in keywords:
    print("word: " + word)
    url = "https://apnews.com/search?q=" + word + "%20&s=0&p="
    df_per_keyword = news_webscrapper(word, url, "div", "PagePromo", "div", "PagePromo-description", "div", "PagePromo-title")
    dfs_with_all_keywords = pd.concat([dfs_with_all_keywords, df_per_keyword], ignore_index=True) #index thing makes everything combine well
    #now add a column that mentions news: AP news
    dfs_with_all_keywords["Source Type"] = "News: AP News"


#dfs_with_all_keywords #filter with specfic words
df_ap = dfs_with_all_keywords

#bbc news
dfs_with_all_keywords = pd.DataFrame()
for word in keywords:
    print("word: " + word)
    url = "https://www.bbc.com/search?q=" + str(word) + "&page="
    df_per_keyword = news_webscrapper(word, url, "div", "sc-cdecfb63-0 cJcHVD", "h2", "sc-fa814188-3 ifRknM", "div", "sc-cdecfb63-3 iIFdam")
    dfs_with_all_keywords = pd.concat([dfs_with_all_keywords, df_per_keyword], ignore_index=True) #index thing makes everything combine well
    #now add a column that mentions news: BBC news
    dfs_with_all_keywords["Source Type"] = "News: BBC News"


#dfs_with_all_keywords #filter with specfic words
df_bbc = dfs_with_all_keywords

#pbs news
dfs_with_all_keywords = pd.DataFrame()
for word in keywords:
    print("word: " + word)
    url = "https://www.pbs.org/newshour/search-results?q=" + word + "&pnb="
    df_per_keyword = news_webscrapper(word, url, "li", "search-result", "h4", "search-result__title", "p", "search-result__snippet")
    dfs_with_all_keywords = pd.concat([dfs_with_all_keywords, df_per_keyword], ignore_index=True) #index thing makes everything combine well
    #now add a column that mentions news: BBC news
    dfs_with_all_keywords["Source Type"] = "News: PBS News"

df_pbs= dfs_with_all_keywords

#putting them together
df_news = pd.concat([df_ap, df_bbc], ignore_index=True)
df_news = pd.concat([df_news, df_pbs], ignore_index=True)
df_news.head()

In [ ]:
#Magazines
#WARNING: this section is very slow, try not to run more than is needed
#it also is very sensitive to changes, try to make as few edits as possible

number_of_clicks = 5 #this is how many times you click the "more stories" button
#for a quick test of the code, set to 1

#Vogue
Headlines=[]
Bylines=[]
Dates=[]
Keyword=[]

for key in keywords:
    keyword = key.upper()
    url = "https://www.vogue.com/search?q=" + keyword +"&sort=score+desc"
    soup = link2soup(url)

    if soup.find('h1', class_="BaseWrap-sc-gzmcOU BaseText-eqOrNE SectionHeaderHed-jvdKHz deqABF iqLDlA dabhPm section-header__hed").get_text() == 'No stories found for your search':
        print("oops can't find anything")
    else:
        soup = click_button_vogue(url, number_of_clicks) #DO NOT REMOVE, is needed to load page properly for scraping
        for headline in soup.find_all('h3'):
            Headlines.append(headline.get_text())
        for byline in soup.find_all('span', itemprop="name", class_="BylineNamesWrapper-jrdaOa fXeqQN"):
            Bylines.append(byline.get_text().replace("By ", ""))
        for date in soup.find_all('time', class_="BaseWrap-sc-gzmcOU BaseText-eqOrNE SummaryItemBylinePublishDate-czeIQl deqABF mdLVF gtAwEp summary-item__publish-date"):
            Dates.append(date.get_text())
            Keyword.append(key)

df = pd.DataFrame({"Headlines":Headlines, "Bylines":Bylines, "Dates":Dates, "Keyword":Keyword})
df["Source"]="Vogue"
df["Source Type"]="Magazine"
df_vogue=df

#TeenVogue
df_teenvogue = teenvogue_scraper("teenvogue", keywords, number_of_clicks,
                                  ["h1", "BaseWrap-sc-gzmcOU BaseText-eqOrNE SectionHeaderHed-jvdKHz deqABF ddMSAi hZnCQy section-header__hed",
                                   "h2", "SummaryItemHedBase-hnYOxl fxKPGc summary-item__hed",
                                   "span", "BylineNamesWrapper-jrdaOa fXeqQN",
                                   "time", "BaseWrap-sc-gzmcOU BaseText-eqOrNE SummaryItemBylinePublishDate-czeIQl deqABF eVEhbw gtAwEp summary-item__publish-date"])

In [ ]:
df_magazines = pd.concat(df_vogue, df_teenvogue)
df_magazines.head()

In [ ]:
#combining dataframes and saving them as csv files

#saving the separate dataframes
df_journals.to_csv('journal_data.csv', index=False)
df_news.to_csv('news_data.csv', index=False)
df_magazines.to_csv('magazine_data.csv', index=False)

In [ ]:
#making one consistant dataframe and saving all of the data
#we did not do a great job keeping all of the column names consistant across the different dataframes
#so this line will not run
#I need you to find the columns that all the dataframes have and rename them so that they all line up
#df_renamed = df.rename(columns={'old_col_A': 'new_col_X', 'old_col_B': 'new_col_Y'})

df_all_sources = pd.concat([df_journals, df_news, df_magazines], ignore_index=True)
df_all_sources.head()
df_all_sources.to_csv('webscraped_data.csv', index=False)